In [1]:
seed_value= 0

import os
os.environ['PYTHONHASHSEED']=str(seed_value)

import random
random.seed(seed_value)

import numpy as np
np.random.seed(seed_value)

import tensorflow as tf

from keras import backend as K
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passi

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import sqrt
import pickle
import math

#Scikit-Learn Importing
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import make_scorer
from sklearn.svm import SVR

#Import PyWavelets for WaveSmoothing
import pywt
import mad
from statsmodels import robust

#Import Keras
import keras
from keras.layers import Input, Dense, BatchNormalization, Dropout, LSTM, RepeatVector, TimeDistributed
from keras.models import Model, Sequential
from keras.optimizers import Adam, SGD, Adadelta
from tqdm import tqdm
from keras.callbacks import History 
history = History()
from keras import backend as K

#Import Gridsearch
from hypopt import GridSearch

#Import Capstone Functions
from Capstone_Functions import *
cf = CapstoneFunctions()

In [3]:
Monthly = pd.read_pickle('data/Monthly.pkl')
Monthly2 = pd.read_pickle('data/MonthlyReturns.pkl')

In [4]:
WOMonthly = Monthly.iloc[:,14:15].copy()
WOMonthly.head()

,WOMonthly
date,
1990-12-03,0.052244
1991-01-01,0.011277
1991-02-01,0.015239
1991-03-01,0.096461
1991-04-01,0.026566


In [5]:
FIPMonthly = Monthly2.iloc[:,16:17].copy()
FIPMonthly.head()

,FIPMonthly
date,
1990-12-03,0.034620
1991-01-01,0.022021
1991-02-01,0.013218
1991-03-01,0.089188
1991-04-01,0.031861


In [6]:
FIPMonthly.shape

(356, 1)

In [7]:
#Create Train_Test_Split for High and Low Momentum returns with a 60 Month Lookback
WOXdata, WOYdata, WOXtrain, WOYtrain, WOXtest, WOYtest, WOPredictedX, WOForecastX, WOX, WOY = \
cf.Train_Test_Split(WOMonthly, 0, 100, 12, scaler = False)

FIPXdata, FIPYdata, FIPXtrain, FIPYtrain, FIPXtest, FIPYtest, FIPPredictedX, FIPForecastX, FIPX, FIPY = \
cf.Train_Test_Split(FIPMonthly, 0, 100, 12, scaler = False)

In [8]:
def waveletSmooth(x, wavelet="haar", level=2, DecLvl=2):
    # calculate the wavelet coefficients
    coeff = pywt.wavedec(x, wavelet, mode="per", level=DecLvl)
    # calculate a threshold
    sigma = robust.mad(coeff[-level])
    uthresh = sigma * np.sqrt( 2*np.log( len( x )))
    coeff[1:] = (pywt.threshold( i, value=uthresh, mode="soft" ) for i in coeff[1:])
    # reconstruct the signal using the thresholded coefficients
    y = pywt.waverec(coeff, wavelet, mode="per")
    return y

In [9]:
wav_wo = waveletSmooth(WOMonthly.iloc[:,0])
wav_wo = wav_wo.reshape(len(wav_wo),1)

wav_fip = waveletSmooth(FIPMonthly.iloc[:,0])
wav_fip = wav_fip.reshape(len(wav_fip),1)

In [10]:
    def Train_Test_Split2(Data1, Hi_or_Lo, Window, Lookback): 

        """ 
        Import Monthly data with Date as Index. Select which portfolio to train test split. Set number of
        lookback months and whether to use MinMaxScaler.

        Example:
        HiXdata, HiYdata, HiXtrain, HiYtrain, HiXtest, HiYtest, HiPredictedX, HiForecastX, HiX, HiY = \
        Train_Test_Split(Monthly, 'Hi', 60, 12, scaler = False)

        """ 


        #If High or Low Momentum Portfolio
        if Hi_or_Lo == 'Hi':
            Port = 0
        else:
            Port =  0

        TrainSplit = int(Window*0.9)
        TestSplit = int(Window*0.1)

        #Iterate Through Data and Creat Numpy arrays with 12 Months of \ 
        #lagged data for X and 13th Month for Y
        tmpX=[]
        tmpY=[]
        for A in range(len(Data1)-Lookback):
            tmp=Data1[A:(A + Lookback),Port]
            tmpX.append(tmp)
            tmpY.append(Data1[(A + Lookback),Port])
        Xdata = np.array(tmpX)
        Ydata = np.array(tmpY)

        tmpForeX=[]
        for A in range(len(Data1)-Lookback):
            tmpFore=Data1[A+1:(A+1 + Lookback),Port]
            tmpForeX.append(tmpFore)
        XdataFore = np.array(tmpForeX)

        Ydata = Ydata.reshape(len(Ydata),1)

        Shape = int(Xdata.shape[0])
        Shape2 = int(XdataFore.shape[0])


        #Create Empty Numpy Arrays
        Xtrain=np.ones(shape=(Shape-Window,TrainSplit,12))
        Ytrain=np.ones(shape=(Shape-Window,TrainSplit,1))
        Xtest=np.ones(shape=(Shape-Window,TestSplit,Lookback))
        Ytest=np.ones(shape=(Shape-Window,TestSplit,1))
        PredictedX=np.ones(shape=(Shape-Window,1,Lookback))
        ForecastX=np.ones(shape=(Shape2-Window,1,Lookback))
        X = np.ones(shape=(Shape-Window,Window,Lookback))
        Y = np.ones(shape=(Shape-Window,Window,1))

        #Fill Numpy Arrays with data
        for B in range(Shape-Window):
            Xtrain[B,:,:] = Xdata[B:B+TrainSplit,:]
            Ytrain[B,:,:] = Ydata[B:B+TrainSplit]
            Xtest[B,:,:] = Xdata[B+TrainSplit:B+Window,:]
            Ytest[B,:,:] = Ydata[B+TrainSplit:B+Window]
            PredictedX[B,:,:] = Xdata[B+Window,:]
            ForecastX[B,:,:] = XdataFore[B+Window,:]
            X[B,:,:] = Xdata[B:B+Window,:]
            Y[B,:,:] = Ydata[B:B+Window]

        return Xdata, Ydata, Xtrain, Ytrain, Xtest, Ytest, PredictedX, ForecastX, X, Y

In [11]:
#Create Train_Test_Split for High and Low Momentum returns with a 60 Month Lookback
W_WO_Xdata, W_WO_Ydata, W_WO_Xtrain, W_WO_Ytrain, W_WO_Xtest, W_WO_Ytest, W_WO_ForecastX, \
W_WO_ForecastX, W_WO_X, W_WO_Y = \
Train_Test_Split2(wav_wo, 'Hi', 100, 12)

#Create Train_Test_Split for High and Low Momentum returns with a 60 Month Lookback
W_FIP_Xdata, W_FIP_Ydata, W_FIP_Xtrain, W_FIP_Ytrain, W_FIP_Xtest, W_FIP_Ytest, W_FIP_ForecastX,\
W_FIP_ForecastX, W_FIP_X, W_FIP_Y = \
Train_Test_Split2(wav_fip, 'Hi', 100, 12)

# LSTM

In [12]:
epochs = 200
batch = 100

In [13]:
def LSTM_Reshape(LSTM_Shape):
    LSTM_Shape = LSTM_Shape.reshape((LSTM_Shape.shape[0],LSTM_Shape.shape[1],1))
    return LSTM_Shape

def LSTM_RNN(Xtrain, Ytrain, Xtest, Ytest, Xtrue):
    keras.backend.clear_session()
    LSTM_ = Sequential()
    LSTM_.add(LSTM(8, input_shape = (12,1), return_sequences = True))
    LSTM_.add(Dropout(0.2))
    LSTM_.add(LSTM(8, return_sequences = True))
    LSTM_.add(Dropout(0.2))
    LSTM_.add(LSTM(8, return_sequences = False))
    LSTM_.add(Dropout(0.2))
    LSTM_.add(Dense(1, activation = 'linear'))
    LSTM_.compile(loss='mse', optimizer = Adam(lr = 0.05))
    LSTM_.fit(Xtrain, Ytrain, batch_size = batch, epochs = epochs,
       validation_data = (Xtest, Ytest), shuffle = False, verbose = 0, stateful = True)
    Prediction_ = LSTM_.predict(Xtrue)
    history = LSTM_.fit(Xtrain, Ytrain, batch_size = batch, epochs = epochs,
       validation_data = (Xtest, Ytest), shuffle = False, verbose = 0, stateful = True)
    Prediction_ = LSTM_.predict(Xtrue)
    
    print(history.history.keys())
   
    plt.subplot(212)  
    plt.plot(history.history['loss'])  
    plt.plot(history.history['val_loss'])  
    plt.title('model loss')  
    plt.ylabel('loss')  
    plt.xlabel('epoch')  
    plt.legend(['train', 'test'], loc='upper left')  
    plt.show()
    
    return (Prediction_)

def predictLSTM(Xtrain, Ytrain, Xtest, Ytest, XFore):
    Final_Prediction = np.zeros(shape=(244,1))
    for i in range(Final_Prediction.shape[0]):
        Xtrain_Reshape = LSTM_Reshape(Xtrain[i])
        Xtest_Reshape = LSTM_Reshape(Xtest[i])
        XFore_Reshape = LSTM_Reshape(XFore[i])
        Final_Prediction[i] = LSTM_RNN(Xtrain_Reshape, Ytrain[i], Xtest_Reshape, Ytest[i], XFore_Reshape)
        print('Prediction',i+1,'of',244)
    return Final_Prediction

In [14]:
pred_FIP_LSTM = \
predictLSTM(W_FIP_Xtrain, FIPYtrain, W_FIP_Xtest, FIPYtest, W_FIP_ForecastX)

Instructions for updating:
Colocations handled automatically by placer.


TypeError: Unrecognized keyword arguments: {'stateful': True}

In [ ]:
LSTM_FIP_Forecast = pd.DataFrame(pred_FIP_LSTM)

In [ ]:
LSTM_FIP_Forecast.to_pickle('data/LSTM_FIP_Forecast.pkl')

In [ ]:
stop